# Fraud Detection — Построение модели

## 🎯 Цель

- Балансировать классы с помощью SMOTE
- Построить модель XGBoost
- Оценить её качество

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

In [6]:
df = pd.read_csv('/content/creditcard.csv')

X = df.drop(['Class'], axis = 1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.2,
                                                    stratify=y,
                                                    random_state = 42)

In [7]:
smote = SMOTE(random_state = 42)

x_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

print("До SMOTE:", y_train.value_counts())
print("После SMOTE:", y_train_bal.value_counts())

До SMOTE: Class
0    227451
1       394
Name: count, dtype: int64
После SMOTE: Class
0    227451
1    227451
Name: count, dtype: int64


In [8]:
model = XGBClassifier(use_label_encoder = False, random_state = 42, eval_metrics = 'logloss')
model.fit(x_train_bal, y_train_bal)

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [13:33:16] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "eval_metrics", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None,
              eval_metrics='logloss', feature_types=None, feature_weights=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, ...)

In [9]:
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.79      0.85      0.82        98

    accuracy                           1.00     56962
   macro avg       0.90      0.92      0.91     56962
weighted avg       1.00      1.00      1.00     56962

ROC-AUC: 0.9831145633548862


## 📈 Вывод

- За счёт SMOTE модель стала гораздо лучше определять класс "1" (мошенничество).
- Recall по классу 1 должен заметно вырасти.
- ROC-AUC выше baseline, что говорит об адекватной обучаемости.


In [10]:
import joblib
joblib.dump(model, "xgb_model.pkl")

['xgb_model.pkl']